In [5]:
!python3.8 -m pip install arcgis

In [6]:
from arcgis.learn import SiamMask

2022-11-15 09:35:07.964233: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-15 09:35:08.301116: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-15 09:35:08.301163: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-11-15 09:35:08.301167: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2022-11-15 09:35:08.633312: W tensorflow/stream_executor/platfo

torch not available



Exception: Traceback (most recent call last):

  File "/home/bikram/.local/lib/python3.8/site-packages/arcgis/learn/models/_siammask_utils.py", line 3, in <module>
    from fastai.basic_train import LearnerCallback

ModuleNotFoundError: No module named 'fastai'


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "/home/bikram/.local/lib/python3.8/site-packages/arcgis/learn/models/_siammask.py", line 15, in <module>
    from ._siammask_utils import (

  File "/home/bikram/.local/lib/python3.8/site-packages/arcgis/learn/models/_siammask_utils.py", line 20, in <module>
    traceback.format_exception(type(e), e, e.__traceback__)

NameError: name 'traceback' is not defined
 


 

In [3]:
!python3.8 -m pip install https://github.com/fastai/fastai1/archive/master.zip


     / 132.1 MB 63 kB/ss
     |████████████████████████████████| 355 kB 16.8 MB/s eta 0:00:01
     |████████████████████████████████| 381 kB 203.4 MB/s eta 0:00:01
  Created wheel for fastai: filename=fastai-1.0.64.dev0-py3-none-any.whl size=241877 sha256=514d041728ae7948ad1dcf2f656a9f9177da309c36af86936d6883676d352d5c
  Stored in directory: /tmp/pip-ephem-wheel-cache-wenxt85y/wheels/7d/12/0a/6a9635e4c0d57914b3ea11de44893ce877696a2e3e400836b2
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19189 sha256=375f7348a4fc97c50ef368a9511302a7db0007540574e772c9c4f846e4ccc257
  Stored in directory: /home/bikram/.cache/pip/wheels/b9/b1/68/cb4feab29709d4155310d29a421389665dcab9eb3b679b527b
Successfully built fastai nvidia-ml-py3


In [7]:
ot = SiamMask()

[INFO] Downloading pretrained weights for SiamMask in /home/bikram/.cache/siammask_checkpoint/SiamMask_DAVIS.pth


/home/bikram/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:104: UserWarning: 
NVIDIA GeForce RTX 3090 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70 sm_75.
If you want to use the NVIDIA GeForce RTX 3090 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [8]:
import numpy as np
import cv2
import json
cap = cv2.VideoCapture(r"train_videos/000/Right.mp4")



annotation_path = "../train_annotations/000.json"
f = open(annotation_path)
data = json.load(f)
data_required = data['sequence']






initialized = False
while(True):
    ret, frame = cap.read()
    if ret is False:
        break
    if initialized:
        state = ot.update(frame)  ## Update the track location in the frame
        for track in state:
            mask = track.mask
            frame[:, :, 2] = (mask > 0) * 255 + (mask == 0) * frame[:, :, 2]
            cv2.polylines(frame, [np.int0(track.location).reshape((-1, 1, 2))], True, (w, 255, h), 1)
            
    cv2.imshow('frame',frame)
    key = cv2.waitKey(1)
    if key & 0xFF == ord('q'):
        break
    else:
        x=int(data_required[x]['TgtXPos_LeftUp'])
        y=int(data_required[x]['TgtYPos_LeftUp'])
        w=int(data_required[x]['TgtWidth'])
        h=int(data_required[x]['TgtHeight'])
        state = ot.init(frame, [[x,y,w,h]]) ## Initialize the track in the frame
        initialized = True

cap.release()
cv2.destroyAllWindows()